<a href="https://colab.research.google.com/github/Yassaadi/Scaling_cnn_emr/blob/main/AWS_EMR_Assaadi_Yassine_1_notebook_052022.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### 4.10.1 Démarrage de la session Spark

In [ ]:
# L'exécution de cette cellule démarre l'application Spark

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
0,application_1683515387863_0001,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

<u>Affichage des informations sur la session en cours et liens vers Spark UI</u> :

In [ ]:
%%info

ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
0,application_1683515387863_0001,pyspark,idle,Link,Link,None,✔


### 4.10.2 Installation des packages

Les packages nécessaires ont été installé via l'étape de **bootstrap** à l'instanciation du serveur.

### 4.10.3 Import des librairies

In [ ]:
import pandas as pd
import numpy as np
import io
import os
import tensorflow as tf
from tensorflow import keras
from PIL import Image
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2, preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras import Model
from pyspark.sql.functions import col, pandas_udf, PandasUDFType, element_at, split
from pyspark.ml.feature import VectorAssembler, StandardScaler, PCA
import matplotlib.pyplot as plt

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### 4.10.4 Définition des PATH pour charger les images et enregistrer les résultats

Nous accédons directement à nos **données sur S3** comme si elles étaient **stockées localement**.

In [ ]:
PATH = 's3://yass-p8-data'
PATH_Data = PATH+'/Test'
PATH_Result = PATH+'/Results'
print('PATH:        '+\
      PATH+'\nPATH_Data:   '+\
      PATH_Data+'\nPATH_Result: '+PATH_Result)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

PATH:        s3://yass-p8-data
PATH_Data:   s3://yass-p8-data/Test
PATH_Result: s3://yass-p8-data/Results

### 4.10.5 Traitement des données

#### 4.10.5.1 Chargement des données

In [ ]:
images = spark.read.format("binaryFile") \
  .option("pathGlobFilter", "*.jpg") \
  .option("recursiveFileLookup", "true") \
  .load(PATH_Data)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
images.show(5)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+-------------------+------+--------------------+
|                path|   modificationTime|length|             content|
+--------------------+-------------------+------+--------------------+
|s3://yass-p8-data...|2023-04-26 16:30:23|125135|[FF D8 FF E0 00 1...|
|s3://yass-p8-data...|2023-04-26 16:30:23|124785|[FF D8 FF E0 00 1...|
|s3://yass-p8-data...|2023-04-26 16:30:23|123514|[FF D8 FF E0 00 1...|
|s3://yass-p8-data...|2023-04-26 16:30:23|122958|[FF D8 FF E0 00 1...|
|s3://yass-p8-data...|2023-04-26 16:30:23|122807|[FF D8 FF E0 00 1...|
+--------------------+-------------------+------+--------------------+
only showing top 5 rows

<u>Je ne conserve que le **path** de l'image et j'ajoute <br />
    une colonne contenant les **labels** de chaque image</u> :

In [ ]:
images = images.withColumn('label', element_at(split(images['path'], '/'),-2))
print(images.printSchema())
print(images.select('path','label').show(5,False))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- path: string (nullable = true)
 |-- modificationTime: timestamp (nullable = true)
 |-- length: long (nullable = true)
 |-- content: binary (nullable = true)
 |-- label: string (nullable = true)

None
+---------------------------------------------+-----------+
|path                                         |label      |
+---------------------------------------------+-----------+
|s3://yass-p8-data/Test/apple_hit_1/r0_115.jpg|apple_hit_1|
|s3://yass-p8-data/Test/apple_hit_1/r0_119.jpg|apple_hit_1|
|s3://yass-p8-data/Test/apple_hit_1/r0_107.jpg|apple_hit_1|
|s3://yass-p8-data/Test/apple_hit_1/r0_143.jpg|apple_hit_1|
|s3://yass-p8-data/Test/apple_hit_1/r0_111.jpg|apple_hit_1|
+---------------------------------------------+-----------+
only showing top 5 rows

None

#### 4.10.5.2 Préparation du modèle

In [ ]:
model = MobileNetV2(weights='imagenet',
                    include_top=True,
                    input_shape=(224, 224, 3))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

14536120/14536120 [==============================] - 0s 0us/step

In [ ]:
new_model = Model(inputs=model.input,
                  outputs=model.layers[-2].output)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
brodcast_weights = sc.broadcast(new_model.get_weights())

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
new_model.summary()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 Conv1 (Conv2D)                 (None, 112, 112, 32  864         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 bn_Conv1 (BatchNormalization)  (None, 112, 112, 32  128         ['Conv1[0][0]']                  
                                )                                                             

In [ ]:
def model_fn():
    """
    Returns a MobileNetV2 model with top layer removed 
    and broadcasted pretrained weights.
    """
    model = MobileNetV2(weights='imagenet',
                        include_top=True,
                        input_shape=(224, 224, 3))
    for layer in model.layers:
        layer.trainable = False
    new_model = Model(inputs=model.input,
                  outputs=model.layers[-2].output)
    new_model.set_weights(brodcast_weights.value)
    return new_model

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

#### 4.10.5.3 Définition du processus de chargement des images <br/> et application de leur featurisation à travers l'utilisation de pandas UDF

In [ ]:
def preprocess(content):
    """
    Preprocesses raw image bytes for prediction.
    """
    img = Image.open(io.BytesIO(content)).resize([224, 224]) #Open image from binary objects content and resize it
    arr = img_to_array(img) # Convert img to to array
    return preprocess_input(arr) #Reshape to (224,224,3)

def featurize_series(model, content_series):
    """
    Featurize a pd.Series of raw images using the input model.
    :return: a pd.Series of image features
    """
    input = np.stack(content_series.map(preprocess))
    preds = model.predict(input)
    # For some layers, output features will be multi-dimensional tensors.
    # We flatten the feature tensors to vectors for easier storage in Spark DataFrames.
    output = [p.flatten() for p in preds]
    return pd.Series(output)

@pandas_udf('array<float>', PandasUDFType.SCALAR_ITER)
def featurize_udf(content_series_iter):
    '''
    This method is a Scalar Iterator pandas UDF wrapping our featurization function.
    The decorator specifies that this returns a Spark DataFrame column of type ArrayType(FloatType).

    :param content_series_iter: This argument is an iterator over batches of data, where each batch
                              is a pandas Series of image data.
    '''
    # With Scalar Iterator pandas UDFs, we can load the model once and then re-use it
    # for multiple data batches.  This amortizes the overhead of loading big models.
    model = model_fn() 
    #Weights broadcasting
    model.set_weights(brodcast_weights.value)
    for content_series in content_series_iter:
        yield featurize_series(model, content_series)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

/mnt1/yarn/usercache/livy/appcache/application_1683515387863_0001/container_1683515387863_0001_01_000001/pyspark.zip/pyspark/sql/pandas/functions.py:398: UserWarning: In Python 3.6+ and Spark 3.0+, it is preferred to specify type hints for pandas UDF instead of specifying pandas UDF type which will be deprecated in the future releases. See SPARK-28264 for more details.

#### 4.10.5.4 Exécutions des actions d'extractions de features

In [ ]:
# spark.conf.set("spark.sql.execution.arrow.maxRecordsPerBatch", "1024")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
features_df = images.repartition(24).select(col("path"),
                                            col("label"),
                                            featurize_udf("content").alias("features")
                                           )

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
print(PATH_Result)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

s3://yass-p8-data/Results

In [ ]:
features_df.write.mode("overwrite").parquet(PATH_Result)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### 4.10.6 Chargement des données enregistrées et validation du résultat

In [ ]:
df = pd.read_parquet(PATH_Result, engine='pyarrow')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
df.head()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

                                            path  ...                                           features
0  s3://yass-p8-data/Test/apple_hit_1/r1_299.jpg  ...  [0.88818276, 0.9766362, 1.6814375, 0.0, 0.7473...
1  s3://yass-p8-data/Test/apple_hit_1/r0_167.jpg  ...  [0.23584183, 0.13031594, 1.4356456, 0.0, 0.357...
2   s3://yass-p8-data/Test/apple_hit_1/r1_43.jpg  ...  [0.008909265, 0.20073241, 2.0521822, 0.0, 0.17...
3  s3://yass-p8-data/Test/apple_hit_1/r2_203.jpg  ...  [0.07983717, 0.30655688, 0.12255844, 0.0, 0.51...
4   s3://yass-p8-data/Test/apple_hit_1/r1_63.jpg  ...  [0.16827433, 0.17030035, 0.27944311, 0.0, 0.48...

[5 rows x 3 columns]

In [ ]:
df.loc[0,'features'].shape

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

(1280,)

In [ ]:
df.shape

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

(3110, 3)

In [ ]:
df1 = spark.read.parquet(PATH_Result)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
df1.printSchema()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- path: string (nullable = true)
 |-- label: string (nullable = true)
 |-- features: array (nullable = true)
 |    |-- element: float (containsNull = true)

## 4.11.6 Standard scalering image features

In [ ]:
from pyspark.ml.linalg import VectorUDT, Vectors
from pyspark.sql.functions import udf
from pyspark.sql.types import ArrayType, DoubleType

# define a UDF for converting numpy arrays to PySpark VectorUDT objects
numpy_to_vector = udf(lambda a: Vectors.dense(a), returnType=VectorUDT())

# apply the UDF to the 'features' column to create a new column of VectorUDT objects
df1 = df1.withColumn('features_vector', numpy_to_vector(df1['features']))
df1 = df1.withColumn('label_red', element_at(split(df1['label'], '_'),1))
# drop the original 'features' column
#df = df.drop('features')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
df1.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+---------------+--------------------+--------------------+---------+
|                path|          label|            features|     features_vector|label_red|
+--------------------+---------------+--------------------+--------------------+---------+
|s3://yass-p8-data...|    apple_hit_1|[0.3821219, 0.231...|[0.38212189078330...|    apple|
|s3://yass-p8-data...|    apple_hit_1|[0.7551134, 1.278...|[0.75511342287063...|    apple|
|s3://yass-p8-data...|    apple_hit_1|[0.77546775, 1.28...|[0.77546775341033...|    apple|
|s3://yass-p8-data...|    apple_hit_1|[0.3894314, 0.012...|[0.38943138718605...|    apple|
|s3://yass-p8-data...|    apple_hit_1|[0.0, 0.2023897, ...|[0.0,0.2023897022...|    apple|
|s3://yass-p8-data...|    apple_hit_1|[0.60624766, 0.01...|[0.60624766349792...|    apple|
|s3://yass-p8-data...|cabbage_white_1|[0.0, 0.5918547, ...|[0.0,0.5918546915...|  cabbage|
|s3://yass-p8-data...|    apple_hit_1|[0.5113373, 0.581...|[0.51133728027343...|    apple|

In [ ]:
scaler = StandardScaler(
    inputCol = 'features_vector', 
    outputCol = 'scaledFeatures',
    withMean = True,
    withStd = True
).fit(df1)

# when we transform the dataframe, the old
# feature will still remain in it
df_scaled = scaler.transform(df1)
df_scaled.show(6)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+-----------+--------------------+--------------------+---------+--------------------+
|                path|      label|            features|     features_vector|label_red|      scaledFeatures|
+--------------------+-----------+--------------------+--------------------+---------+--------------------+
|s3://yass-p8-data...|apple_hit_1|[0.3821219, 0.231...|[0.38212189078330...|    apple|[0.04212617474026...|
|s3://yass-p8-data...|apple_hit_1|[0.7551134, 1.278...|[0.75511342287063...|    apple|[0.91827456645018...|
|s3://yass-p8-data...|apple_hit_1|[0.77546775, 1.28...|[0.77546775341033...|    apple|[0.96608641318808...|
|s3://yass-p8-data...|apple_hit_1|[0.3894314, 0.012...|[0.38943138718605...|    apple|[0.05929601096048...|
|s3://yass-p8-data...|apple_hit_1|[0.0, 0.2023897, ...|[0.0,0.2023897022...|    apple|[-0.8554692161614...|
|s3://yass-p8-data...|apple_hit_1|[0.60624766, 0.01...|[0.60624766349792...|    apple|[0.56859237681661...|
+--------------------+------

In [ ]:
df_scaled.printSchema()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- path: string (nullable = true)
 |-- label: string (nullable = true)
 |-- features: array (nullable = true)
 |    |-- element: float (containsNull = true)
 |-- features_vector: vector (nullable = true)
 |-- label_red: string (nullable = true)
 |-- scaledFeatures: vector (nullable = true)

## 3.10 Performing PCA

In [ ]:
n_components = 20
pca = PCA(
    k = n_components, 
    inputCol = 'scaledFeatures', 
    outputCol = 'pcaFeatures'
).fit(df_scaled)
print('Cumulative variance', pca.explainedVariance.toArray().sum())

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Cumulative variance 0.5376154532383266

In [ ]:
variances = pca.explainedVariance.toArray()
# Compute the cumulative explained variance
cumulative_variances = np.cumsum(variances)

# Plot the explained variance ratios
plt.bar(range(1, n_components + 1), variances, label='Individual explained variance')
plt.plot(range(1, n_components + 1), cumulative_variances,'-s', label='Cumulative explained variance', color='red')
plt.xlabel('Number of principal components')
plt.ylabel('Explained variance ratio')
plt.legend(loc='best')
plt.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
n_components = 2
Pca = PCA(
    k = n_components, 
    inputCol = 'scaledFeatures', 
    outputCol = 'pcaFeatures'
).fit(df_scaled)

df_pca = Pca.transform(df_scaled)
print('Explained Variance Ratio', Pca.explainedVariance.toArray())
df_pca.show(6)
X_pca = df_pca.rdd.map(lambda row: row.pcaFeatures).collect()
X_pca = np.array(X_pca)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Explained Variance Ratio [0.1133648  0.06571123]
+--------------------+-----------+--------------------+--------------------+---------+--------------------+--------------------+
|                path|      label|            features|     features_vector|label_red|      scaledFeatures|         pcaFeatures|
+--------------------+-----------+--------------------+--------------------+---------+--------------------+--------------------+
|s3://yass-p8-data...|apple_hit_1|[0.3821219, 0.231...|[0.38212189078330...|    apple|[0.04212617474026...|[1.56348457055567...|
|s3://yass-p8-data...|apple_hit_1|[0.7551134, 1.278...|[0.75511342287063...|    apple|[0.91827456645018...|[12.0309400335016...|
|s3://yass-p8-data...|apple_hit_1|[0.77546775, 1.28...|[0.77546775341033...|    apple|[0.96608641318808...|[7.84802238865728...|
|s3://yass-p8-data...|apple_hit_1|[0.3894314, 0.012...|[0.38943138718605...|    apple|[0.05929601096048...|[-1.4119891840638...|
|s3://yass-p8-data...|apple_hit_1|[0.0, 0.202389

In [ ]:
# change default style figure and font size
def plot_pca(X, y):
    """Plot of the 2-dimensional image features"""
    labels = np.array(df_pca.select('label').rdd.flatMap(lambda x: x).collect())
    u_labels = np.unique(labels)

    for i in u_labels:
        plt.scatter(X[labels == i , 0] , X[labels == i , 1] , label = f'{i}')
    plt.title(f'PCA Scatter plot', fontsize=14 )
    plt.legend(loc='upper right')# bbox_to_anchor=(0.5, -0.2), ncol=len(u_labels)) #,
    plt.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
plot_pca(X_pca, "label_red")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Save PCA dataframe

In [ ]:
PATH_pca = PATH+'/Results/PcaFeatures'
df_pca.write.mode("overwrite").parquet(PATH_pca)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Nous venons de valider le processus sur un jeu de données allégé en local <br />
où nous avons simulé un cluster de machines en répartissant la charge de travail <br />
sur différents cœurs de processeur au sein d'une même machine.

Nous allons maintenant généraliser le processus en déployant notre solution <br />
sur un réel cluster de machines et nous travaillerons désormais sur la totalité <br />
des 22819 images de notre dossier "Test".

<u>On peut également constater la présence des fichiers <br />
    au format "**parquet**" sur le **serveur S3**</u> :

![Affichage des résultats sur S3](https://github.com/Yassaadi/Scaling_cnn_emr/blob/main/img/S3_Results.png?raw=1)

## 4.11 Suivi de l'avancement des tâches avec le Serveur d'Historique Spark

Il est possible de voir l'avancement des tâches en cours <br />
avec le **serveur d'historique Spark**.

![Accès au serveur d'historique spark](https://github.com/Yassaadi/Scaling_cnn_emr/blob/main/img/EMR_serveur_historique_spark_acces.png?raw=1)

**Il est également possible de revenir et d'étudier les tâches <br />
qui ont été réalisé, afin de debugger, optimiser les futurs <br />
tâches à réaliser.**

<u>Lorsque la commande "**features_df.write.mode("overwrite").parquet(PATH_Result)**" <br />
était en cours, nous pouvions observer son état d'avancement</u> :

![Progression execution script](https://github.com/Yassaadi/Scaling_cnn_emr/blob/main/img/EMR_jupyterhub_avancement.png?raw=1)

<u>Le **serveur d'historique Spark** nous permet une vision beaucoup plus précise <br />
de l'exécution des différentes tâche sur les différentes machines du cluster</u> :

![Suivi des tâches spark](https://github.com/Yassaadi/Scaling_cnn_emr/blob/main/img/EMR_SHSpark_01.png?raw=1)

On peut également constater que notre cluster de calcul a mis <br />
un tout petit peu **moins de 8 minutes** pour traiter les **22 688 images**.

![Temps de traitement](https://github.com/Yassaadi/Scaling_cnn_emr/blob/main/img/EMR_SHSpark_02.png?raw=1)


## 4.12 Résiliation de l'instance EMR

Notre travail est maintenant terminé. <br />
Le cluster de machines EMR est **facturé à la demande**, <br />
et nous continuons d'être facturé même lorsque <br />
les machines sont au repos.<br />
Pour **optimiser la facturation**, il nous faut <br />
maintenant **résilier le cluster**.

<u>Je réalise cette commande depuis l'interface AWS</u> :

1. Commencez par **désactiver le tunnel ssh dans FoxyProxy** pour éviter des problèmes de **timeout**.
![Désactivation de FoxyProxy](https://github.com/Yassaadi/Scaling_cnn_emr/blob/main/img/EMR_foxyproxy_desactivation.png?raw=1)
2. Cliquez sur "**Résilier**"
![Cliquez sur Résilier](https://github.com/Yassaadi/Scaling_cnn_emr/blob/main/img/EMR_resiliation_01.png?raw=1)
3. Confirmez la résiliation
![Confirmez la résiliation](https://github.com/Yassaadi/Scaling_cnn_emr/blob/main/img/EMR_resiliation_02.png?raw=1)
4. La résiliation prend environ **1 minute**
![Résiliation en cours](https://github.com/Yassaadi/Scaling_cnn_emr/blob/main/img/EMR_resiliation_03.png?raw=1)
5. La résiliation est effectuée
![Résiliation terminée](https://github.com/Yassaadi/Scaling_cnn_emr/blob/main/img/EMR_resiliation_04.png?raw=1)

## 4.13 Cloner le serveur EMR (si besoin)

Si nous devons de nouveau exécuter notre notebook dans les mêmes conditions, <br />
il nous suffit de **cloner notre cluster** et ainsi en obtenir une copie fonctionnelle <br />
sous 15/20 minutes, le temps de son instanciation.

<u>Pour cela deux solutions</u> :
1. <u>Depuis l'interface AWS</u> :
 1. Cliquez sur "**Cloner**"
   ![Cloner un cluster](https://github.com/Yassaadi/Scaling_cnn_emr/blob/main/img/EMR_cloner_01.png?raw=1)
 2. Dans notre cas nous ne souhaitons pas inclure d'étapes
   ![Ne pas inclure d'étapes](https://github.com/Yassaadi/Scaling_cnn_emr/blob/main/img/EMR_cloner_02.png?raw=1)
 3. La configuration du cluster est recréée à l’identique. <br />
    On peut revenir sur les différentes étapes si on souhaite apporter des modifications<br />
    Quand tout est prêt, cliquez sur "**Créer un cluster**"
  ![Vérification/Modification/Créer un cluster](https://github.com/Yassaadi/Scaling_cnn_emr/blob/main/img/EMR_cloner_03.png?raw=1)
2. <u>En ligne de commande</u> (avec AWS CLI d'installé et de configuré et en s'assurant <br />
   de s'attribuer les droits nécessaires sur le compte AMI utilisé)
 1. Cliquez sur "**Exporter AWS CLI**"
 ![Exporter AWS CLI](https://github.com/Yassaadi/Scaling_cnn_emr/blob/main/img/EMR_cloner_cli_01.png?raw=1)
 2. Copier/Coller la commande **depuis un terminal**
 ![Copier Coller Commande](https://github.com/Yassaadi/Scaling_cnn_emr/blob/main/img/EMR_cloner_cli_02.png?raw=1)

## 4.14 Arborescence du serveur S3 à la fin du projet

<u>Pour information, voici **l'arborescence complète de mon bucket S3 p8-data** à la fin du projet</u> : <br />
*Par soucis de lisibilité, je ne liste pas les 131 sous dossiers du répertoire "Test"*

1. Results/_SUCCESS
1. Results/part-00000-2cc36f38-19ef-4d8a-a0d1-5ddb309b3894-c000.snappy.parquet
1. Results/part-00001-2cc36f38-19ef-4d8a-a0d1-5ddb309b3894-c000.snappy.parquet
1. Results/part-00002-2cc36f38-19ef-4d8a-a0d1-5ddb309b3894-c000.snappy.parquet
1. Results/part-00003-2cc36f38-19ef-4d8a-a0d1-5ddb309b3894-c000.snappy.parquet
1. Results/part-00004-2cc36f38-19ef-4d8a-a0d1-5ddb309b3894-c000.snappy.parquet
1. Results/part-00005-2cc36f38-19ef-4d8a-a0d1-5ddb309b3894-c000.snappy.parquet
1. Results/part-00006-2cc36f38-19ef-4d8a-a0d1-5ddb309b3894-c000.snappy.parquet
1. Results/part-00007-2cc36f38-19ef-4d8a-a0d1-5ddb309b3894-c000.snappy.parquet
1. Results/part-00008-2cc36f38-19ef-4d8a-a0d1-5ddb309b3894-c000.snappy.parquet
1. Results/part-00009-2cc36f38-19ef-4d8a-a0d1-5ddb309b3894-c000.snappy.parquet
1. Results/part-00010-2cc36f38-19ef-4d8a-a0d1-5ddb309b3894-c000.snappy.parquet
1. Results/part-00011-2cc36f38-19ef-4d8a-a0d1-5ddb309b3894-c000.snappy.parquet
1. Results/part-00012-2cc36f38-19ef-4d8a-a0d1-5ddb309b3894-c000.snappy.parquet
1. Results/part-00013-2cc36f38-19ef-4d8a-a0d1-5ddb309b3894-c000.snappy.parquet
1. Results/part-00014-2cc36f38-19ef-4d8a-a0d1-5ddb309b3894-c000.snappy.parquet
1. Results/part-00015-2cc36f38-19ef-4d8a-a0d1-5ddb309b3894-c000.snappy.parquet
1. Results/part-00016-2cc36f38-19ef-4d8a-a0d1-5ddb309b3894-c000.snappy.parquet
1. Results/part-00017-2cc36f38-19ef-4d8a-a0d1-5ddb309b3894-c000.snappy.parquet
1. Results/part-00018-2cc36f38-19ef-4d8a-a0d1-5ddb309b3894-c000.snappy.parquet
1. Results/part-00019-2cc36f38-19ef-4d8a-a0d1-5ddb309b3894-c000.snappy.parquet
1. Results/part-00020-2cc36f38-19ef-4d8a-a0d1-5ddb309b3894-c000.snappy.parquet
1. Results/part-00021-2cc36f38-19ef-4d8a-a0d1-5ddb309b3894-c000.snappy.parquet
1. Results/part-00022-2cc36f38-19ef-4d8a-a0d1-5ddb309b3894-c000.snappy.parquet
1. Results/part-00023-2cc36f38-19ef-4d8a-a0d1-5ddb309b3894-c000.snappy.parquet
1. Test/
1. bootstrap-emr.sh
1. jupyter-s3-conf.json
1. jupyter/jovyan/.s3keep
1. jupyter/jovyan/P8_01_Notebook.ipynb
1. jupyter/jovyan/_metadata
1. jupyter/jovyan/e-5OTY4VKPDT21945FF6DN15E35/.aws-editors-workspace-metadata/
1. jupyter/jovyan/e-5OTY4VKPDT21945FF6DN15E35/.aws-editors-workspace-metadata/file-perm.sqlite
1. jupyter/jovyan/e-5OTY4VKPDT21945FF6DN15E35/.aws-editors-workspace-metadata/nbconvert/
1. jupyter/jovyan/e-5OTY4VKPDT21945FF6DN15E35/.aws-editors-workspace-metadata/nbconvert/templates/
1. jupyter/jovyan/e-5OTY4VKPDT21945FF6DN15E35/.aws-editors-workspace-metadata/nbconvert/templates/html/
1. jupyter/jovyan/e-5OTY4VKPDT21945FF6DN15E35/.aws-editors-workspace-metadata/nbconvert/templates/latex/
1. jupyter/jovyan/e-5OTY4VKPDT21945FF6DN15E35/.aws-editors-workspace-metadata/nbsignatures.db
1. jupyter/jovyan/e-5OTY4VKPDT21945FF6DN15E35/.aws-editors-workspace-metadata/notebook_secret
1. jupyter/jovyan/e-5OTY4VKPDT21945FF6DN15E35/.ipynb_checkpoints/
1. jupyter/jovyan/e-5OTY4VKPDT21945FF6DN15E35/.ipynb_checkpoints/Untitled-checkpoint.ipynb
1. jupyter/jovyan/e-5OTY4VKPDT21945FF6DN15E35/.ipynb_checkpoints/Untitled1-checkpoint.ipynb
1. jupyter/jovyan/e-5OTY4VKPDT21945FF6DN15E35/.ipynb_checkpoints/test3-checkpoint.ipynb
1. jupyter/jovyan/e-5OTY4VKPDT21945FF6DN15E35/Untitled.ipynb
1. jupyter/jovyan/e-5OTY4VKPDT21945FF6DN15E35/Untitled1.ipynb
1. jupyter/jovyan/e-5OTY4VKPDT21945FF6DN15E35/test3.ipynb

# 5. Conclusion

Nous avons réalisé ce projet **en deux temps** en tenant <br />
compte des contraintes qui nous ont été imposées.

Nous avons **dans un premier temps développé notre solution en local** <br />
sur une machine virtuelle dans un environnement Linux Ubuntu.

La <u>première phase</u> a consisté à **installer l'environnement de travail Spark**. <br />
**Spark** a un paramètre qui nous permet de travaillé en local et nous permet <br />
ainsi de **simuler du calcul partagé** en considérant <br />
**chaque cœur d'un processeur comme un worker indépendant**.<br />
Nous avons travaillé sur un plus **petit jeu de donnée**, l'idée était <br />
simplement de **valider le bon fonctionnement de la solution**.

Nous avons fait le choix de réaliser du **transfert learning** <br />
à partir du model **MobileNetV2**.<br />
Ce modèle a été retenu pour sa **légèreté** et sa **rapidité d'exécution** <br />
ainsi que pour la **faible dimension de son vecteur en sortie**.

Les résultats ont été enregistrés sur disque en plusieurs <br />
partitions au format "**parquet**".

<u>**La solution a parfaitement fonctionné en mode local**</u>.

La <u>deuxième phase</u> a consisté à créer un **réel cluster de calculs**. <br />
L'objectif était de pouvoir **anticiper une future augmentation de la charge de travail**.

Le meilleur choix retenu a été l'utilisation du prestataire de services **Amazon Web Services** <br />
qui nous permet de **louer à la demande de la puissance de calculs**, <br />
pour un **coût tout à fait acceptable**.<br />
Ce service se nomme **EC2** et se classe parmi les offres **Infrastructure As A Service** (IAAS).

Nous sommes allez plus loin en utilisant un service de plus <br />
haut niveau (**Plateforme As A Service** PAAS)<br />
en utilisant le service **EMR** qui nous permet d'un seul coup <br />
d'**instancier plusieurs serveur (un cluster)** sur lesquels <br />
nous avons pu demander l'installation et la configuration de plusieurs<br />
programmes et librairies nécessaires à notre projet comme **Spark**, <br />
**Hadoop**, **JupyterHub** ainsi que la librairie **TensorFlow**.

En plus d'être plus **rapide et efficace à mettre en place**, nous avons <br />
la **certitude du bon fonctionnement de la solution**, celle-ci ayant été <br />
préalablement validé par les ingénieurs d'Amazon.

Nous avons également pu installer, sans difficulté, **les packages <br />
nécessaires sur l'ensembles des machines du cluster**.

Enfin, avec très peu de modification, et plus simplement encore, <br />
nous avons pu **exécuter notre notebook comme nous l'avions fait localement**.<br />
Nous avons cette fois-ci exécuté le traitement sur **l'ensemble des images de notre dossier "Test"**.

Nous avons opté pour le service **Amazon S3** pour **stocker les données de notre projet**. <br />
S3 offre, pour un faible coût, toutes les conditions dont nous avons besoin pour stocker <br />
et exploiter de manière efficace nos données.<br />
L'espace alloué est potentiellement **illimité**, mais les coûts seront fonction de l'espace utilisé.

Il nous sera **facile de faire face à une monté de la charge de travail** en **redimensionnant** <br />
simplement notre cluster de machines (horizontalement et/ou verticalement au besoin), <br />
les coûts augmenteront en conséquence mais resteront nettement inférieurs aux coûts engendrés <br />
par l'achat de matériels ou par la location de serveurs dédiés.